In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from typing import Optional, Callable
import ipywidgets as wg
from fancy_einsum import einsum

import utils

##  Discrete Fourier Transforms

In [ ]:
def DFT_1d(arr : np.ndarray,  inverse: bool = False) -> np.ndarray:
    """
    Returns the DFT of the array `arr`, using the equation above.
    """
    N = len(arr)
    power_term  = - np.outer(np.arange(N), np.arange(N)) * (2j * np.pi / N)
    unity_arr = np.exp(power_term)
    if not inverse:
        return unity_arr @ arr
    else: 
        return np.linalg.inv(unity_arr) @ arr

def test_DFT_func_2(DFT_1d):

    x = np.array([1, 2 - 1j, -1j, 2j-1])
    y = DFT_1d(x)
    y_expected  = np.array([2, -2 - 2j, -2j, 4+4j])
    np.testing.assert_array_almost_equal(y, y_expected)

utils.test_DFT_func(DFT_1d)
test_DFT_func_2(DFT_1d)

This exercises reminded me that its better to get exponent terms on their own rather and then apply np.exp. 

## Continuous Fourier Transforms

In [ ]:
def integrate_function(func: Callable, x0: float, x1: float, n_samples: int = 1000):
    """
    Calculates the approximation of the Riemann integral of the function `func`, 
    between the limits x0 and x1.

    You should use the Left Rectangular Approximation Method (LRAM).
    """
    step_size = (x1-x0)/n_samples
    intervals = np.arange(x0,x1,step_size)    
    return np.sum(np.array(list(map(func, intervals))))*step_size


utils.test_integrate_function(integrate_function)

In [ ]:
def integrate_product(func1: Callable, func2: Callable, x0: float, x1: float):
    """
    Computes the integral of the function x -> func1(x) * func2(x).
    """
    return integrate_function(lambda x: func1(x)*func2(x), x0, x1)

utils.test_integrate_product(integrate_product)

In [ ]:
def calculate_fourier_series(func: Callable, max_freq: int = 50):
    """
    Calculates the fourier coefficients of a function, 
    assumed periodic between [-pi, pi].

    Your function should return ((a_0, A_n, B_n), func_approx), where:
        a_0 is a float
        A_n, B_n are lists of floats, with n going up to `max_freq`
        func_approx is the fourier approximation, as described above
    """
    
    a_0 = (1/np.pi)*integrate_function(func, -np.pi, np.pi) 
    A_n = [(1/np.pi)*integrate_product(func, lambda x: np.cos(n*x), -np.pi, np.pi)  for n in range(1, 1+max_freq)]
    B_n = [(1/np.pi)*integrate_product(func, lambda x: np.sin(n*x), -np.pi, np.pi)   for n in range(1, 1+max_freq)]
    func_approx =  lambda x: 0.5*a_0 + \
        sum([a*np.cos((n+1)*x) for n, a in enumerate(A_n)]) + \
        sum([b*np.sin((n+1)*x) for n, b in enumerate(B_n)])
    
    return (a_0, A_n, B_n,), np.vectorize(func_approx)

step_func = lambda x: np.exp(np.sin(x)) * (np.sin(3*x) > 0)


utils.create_interactive_fourier_graph(calculate_fourier_series, func = step_func)

Note: Just be careful about what you're writing. Read each line carefully. 

# Basic Neural Network


In [ ]:
NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 40000
LEARNING_RATE = 1e-6

x = np.linspace(-np.pi, np.pi, 2000)
y = TARGET_FUNC(x)

x_cos = np.array([np.cos(n*x) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = np.array([np.sin(n*x) for n in range(1, NUM_FREQUENCIES+1)])

a_0 = np.random.randn()
A_n = np.random.randn(NUM_FREQUENCIES)
B_n = np.random.randn(NUM_FREQUENCIES)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):

    # TODO: compute `y_pred` using your coeffs, and the terms `x_cos`, `x_sin`
    y_pred = 0.5*a_0 + A_n @ x_cos + B_n @ x_sin

    # TODO: compute `loss`, which is the sum of squared error between `y` and `y_pred`
    loss = ((y - y_pred)**2).sum()

    if step % 1000 == 0:
        print(f"{loss = :.2f}")
        coeffs_list.append([a_0, A_n.copy(), B_n.copy()])
        y_pred_list.append(y_pred)

    # TODO: compute gradients of coeffs with respect to `loss`
    grad_pred_y = 2*(y_pred-y)
    grad_coeffs_a_0 = 0.5*grad_pred_y
    grad_coeffs_A_n = x_cos @  grad_pred_y
    grad_coeffs_B_n = x_sin @ grad_pred_y

    # TODO update weights using gradient descent (using the parameter `LEARNING_RATE`)
    a_0 = a_o = LEARNING_RATE*grad_coeffs_a_0
    A_n  = A_n - LEARNING_RATE*grad_coeffs_A_n
    B_n = B_n - LEARNING_RATE*grad_coeffs_B_n

result = utils.visualise_fourier_coeff_convergence(x, y, y_pred_list, coeffs_list)
display(result)

In [17]:
import torch
import utils
from copy import deepcopy

NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, 2000)
y = TARGET_FUNC(x)
list(torch.cos(x))
x_cos = torch.tensor([list(torch.cos(n*x)) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = torch.tensor([list(torch.sin(n*x)) for n in range(1, NUM_FREQUENCIES+1)])

a_0 = torch.randn([1])
A_n = torch.randn(NUM_FREQUENCIES)
B_n = torch.randn(NUM_FREQUENCIES)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):

    # TODO: compute `y_pred` using your coeffs, and the terms `x_cos`, `x_sin`
    y_pred = 0.5*a_0 + A_n @ x_cos + B_n @ x_sin

    # TODO: compute `loss`, which is the sum of squared error between `y` and `y_pred`
    loss = ((y - y_pred)**2).sum()

    if step % 1000 == 0:
        print(f"{loss = :.2f}")
        coeffs_list.append([a_0, deepcopy(A_n), deepcopy(B_n)])
        y_pred_list.append(y_pred)

    # TODO: compute gradients of coeffs with respect to `loss`
    grad_pred_y = 2*(y_pred-y)
    grad_coeffs_a_0 = 0.5*grad_pred_y
    grad_coeffs_A_n = x_cos @  grad_pred_y
    grad_coeffs_B_n = x_sin @ grad_pred_y

    # TODO update weights using gradient descent (using the parameter `LEARNING_RATE`)
    a_0 = a_0 -  LEARNING_RATE*grad_coeffs_a_0
    A_n  = A_n - LEARNING_RATE*grad_coeffs_A_n
    B_n = B_n - LEARNING_RATE*grad_coeffs_B_n

result = utils.visualise_fourier_coeff_convergence(x, y, y_pred_list, coeffs_list)
display(result)

loss = 8928.85
loss = 3670.25
loss = 3570.74
loss = 3565.41


TypeError: unsupported format string passed to Tensor.__format__

In [16]:
import re
import torch
import utils
from copy import deepcopy

NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, 2000)
y = TARGET_FUNC(x)
x_cos = torch.tensor([list(torch.cos(n*x)) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = torch.tensor([list(torch.sin(n*x)) for n in range(1, NUM_FREQUENCIES+1)])

a_0 = torch.randn((), requires_grad=True)
A_n = torch.randn((NUM_FREQUENCIES), requires_grad=True)
B_n = torch.randn((NUM_FREQUENCIES), requires_grad=True)

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):
    
    # TODO: compute `y_pred` using your coeffs, and the terms `x_cos`, `x_sin`
    y_pred = 0.5*a_0 + torch.matmul(A_n, x_cos) + torch.matmul(B_n, x_sin)
    
    # TODO: compute `loss`, which is the sum of squared error between `y` and `y_pred`
    loss = torch.square(y - y_pred).sum()

    if step % 1000 == 0:
        print(f"{loss = :.2f}")
        coeffs_list.append([a_0.detach().item(), A_n.detach().numpy().copy(), B_n.detach().numpy().copy()])
        y_pred_list.append(y_pred.detach())
    
    # TODO: compute gradients of coeffs with respect to `loss`
    loss.backward()

    
    # TODO update weights using gradient descent (using the parameter `LEARNING_RATE`)
    with torch.no_grad():
        a_0 -= LEARNING_RATE * a_0.grad
        A_n  -= LEARNING_RATE * A_n.grad
        B_n -= LEARNING_RATE * B_n.grad

        a_0.grad = None
        A_n.grad = None 
        B_n.grad = None
        

result = utils.visualise_fourier_coeff_convergence(x, y, y_pred_list, coeffs_list)
#display(result)

loss = 3872.38
loss = 251.42
loss = 84.81
loss = 68.17


In [22]:
import re
import torch
import utils
from copy import deepcopy

NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, 2000)
y = TARGET_FUNC(x)
x_cos = torch.tensor([list(torch.cos(n*x)) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = torch.tensor([list(torch.sin(n*x)) for n in range(1, NUM_FREQUENCIES+1)])
input = torch.concat((x_cos, x_sin))
linear_model = torch.nn.Linear(len(x_cos)*2, len(y))

y_pred_list = []
coeffs_list = []

for step in range(TOTAL_STEPS):
    
    # TODO: compute `y_pred` using your coeffs, and the terms `x_cos`, `x_sin`
    y_pred = linear_model(input.T)
    
    # TODO: compute `loss`, which is the sum of squared error between `y` and `y_pred`
    loss = torch.square(y - y_pred).sum()

    if step % 1000 == 0:
        print(f"{loss = :.2f}")
        coeffs_list.append([a_0.detach().item(), A_n.detach().numpy().copy(), B_n.detach().numpy().copy()])
        y_pred_list.append(y_pred.detach())
    
    # TODO: compute gradients of coeffs with respect to `loss`
    loss.backward()

    
    # TODO update weights using gradient descent (using the parameter `LEARNING_RATE`)
    with torch.no_grad():
        for param in linear_model.parameters():
            param -= param.grad*LEARNING_RATE

        linear_model.zero_grad()
        

result = utils.visualise_fourier_coeff_convergence(x, y, y_pred_list, coeffs_list)
#display(result)

loss = 2377978.00
loss = 12567.61
loss = 219.10
loss = 3.99


In [23]:
import re
import torch
import utils
from copy import deepcopy

NUM_FREQUENCIES = 2
TARGET_FUNC = lambda x: 1 * (x > 1)
TOTAL_STEPS = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, 2000)
y = TARGET_FUNC(x)
x_cos = torch.tensor([list(torch.cos(n*x)) for n in range(1, NUM_FREQUENCIES+1)])
x_sin = torch.tensor([list(torch.sin(n*x)) for n in range(1, NUM_FREQUENCIES+1)])
input = torch.concat((x_cos, x_sin))
linear_model = torch.nn.Linear(len(x_cos)*2, len(y))

y_pred_list = []
coeffs_list = []

optim = torch.optim.SGD(linear_model.parameters(), LEARNING_RATE)

for step in range(TOTAL_STEPS):
    
    # TODO: compute `y_pred` using your coeffs, and the terms `x_cos`, `x_sin`
    y_pred = linear_model(input.T)
    
    # TODO: compute `loss`, which is the sum of squared error between `y` and `y_pred`
    loss = torch.square(y - y_pred).sum()

    if step % 1000 == 0:
        print(f"{loss = :.2f}")
        coeffs_list.append([a_0.detach().item(), A_n.detach().numpy().copy(), B_n.detach().numpy().copy()])
        y_pred_list.append(y_pred.detach())
    
    # TODO: compute gradients of coeffs with respect to `loss`
    loss.backward()

    
    # TODO update weights using gradient descent (using the parameter `LEARNING_RATE`)
    with torch.no_grad():
        optim.step()
        linear_model.zero_grad()
        

result = utils.visualise_fourier_coeff_convergence(x, y, y_pred_list, coeffs_list)
#display(result)

loss = 2358885.50
loss = 12776.69
loss = 223.08
loss = 4.07
